# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 28/03/2024

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os
import concurrent.futures
import timeit

import matplotlib
%matplotlib ipympl

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

from xfab.symmetry import Umis

import ImageD11.columnfile
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.peakselect import select_ring_peaks_by_intensity
from ImageD11.sinograms import properties, roi_iradon
from ImageD11.sinograms import geometry
from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_slice_recon, write_h5, read_h5, write_pbp_strain
from ImageD11.grain import grain
from ImageD11 import cImageD11

import ImageD11.nbGui.nb_utils as utils

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'
ds = ImageD11.sinograms.dataset.load(dset_file)

sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# load phases from parameter file

ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# pick a phase
phase_str = 'Si'

In [ ]:
# Import 2D peaks

cf_2d = ds.get_cf_2d()
ds.update_colfile_pars(cf_2d, phase_name=phase_str)

print(f"Read {cf_2d.nrows} 4D peaks")

In [ ]:
# import grainsinos

grainsinos = read_h5(ds.grainsfile, ds, phase_str)
grains = [gs.grain for gs in grainsinos]

In [ ]:
# import slice reconstructions

tensor_map = TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + phase_str)

In [ ]:
# filter 2D peaks by intensity

cf_2d_strong_frac = 0.999

cf_2d_strong = select_ring_peaks_by_intensity(cf_2d, frac=cf_2d_strong_frac, dsmax=cf_2d.ds.max(), doplot=0.5)
print(cf_2d.nrows)
print(cf_2d_strong.nrows)

In [ ]:
y0 = grainsinos[0].recon_y0
recon_shape = grainsinos[0].recons["iradon"].shape

In [ ]:
dtyi = geometry.dty_to_dtyi(cf_2d_strong.dty, ystep=ds.ystep)

cf_2d_strong.addcolumn(dtyi, "dtyi")

In [ ]:
# what peaks could have come from this grain?
# first, assign all peaks to grains 

peak_assign_tol = 0.025

utils.assign_peaks_to_grains(grains, cf_2d_strong, tol=peak_assign_tol)

In [ ]:
per_pixel_ubis = {}
grain_lut = {}

clean_pars = cf_2d_strong.parameters.get_parameters()

for ginc, g in enumerate(tqdm(grains[:])):
    grain_lut[ginc] = g
    
    # work out what 2D peaks this grain claims
    g.mask_2d_strong = cf_2d_strong.grain_id == ginc

    # get the 2D peaks for this grain only
    # for now this will become the columnfile for the grain
    g.cf = cf_2d_strong.copyrows(g.mask_2d_strong)
    
    # work out what pixels in the sample this grain claims
    g.pixel_support_mask = tensor_map.labels == ginc
    g.pixel_support_coords = np.argwhere(g.pixel_support_mask == True)
    
    if len(g.pixel_support_coords) == 0:
        continue
    
    # right now, g.cf is just from the peak assignment
    # could contain peaks that come from the wrong place in the sample
    # solution: combine these 2 masks together
    # isolates 2D peaks that index this grain and come from the grain spatially
    
    # this will mask g.cf based on pixel position
    new_mask_2d_strong = np.zeros(g.cf.nrows).astype(bool)
    
    # iterate through the pixel positions
    for mi, mj, mk in g.pixel_support_coords:
        # get a peak mask associated with this pixel position in the sample
        ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
        mask = geometry.dtyimask_from_recon(ri, rj, g.cf.omega, g.cf.dtyi, recon_shape=recon_shape, ystep=ds.ystep, y0=y0)
        
        # add it to the mask (| is OR)
        new_mask_2d_strong |= mask
    
    g.cf = g.cf.copyrows(new_mask_2d_strong)
    
    gvecs_per_point = {}
    all_tth_old = []
    all_tth_new = []
    all_omega = []
    for mi, mj, mk in g.pixel_support_coords:
        ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
        # get translation in the sample frame
        sx, sy = geometry.recon_to_sample(ri, rj, recon_shape=recon_shape, ystep=ds.ystep)

        # get a peak mask associated with this pixel position in the sample
        pixel_mask = geometry.dtyimask_from_recon(ri, rj, g.cf.omega, g.cf.dtyi, recon_shape=recon_shape, ystep=ds.ystep, y0=y0)
        
        # get the x translation of the peak in the lab reference frame
        
        lx, _ = geometry.sample_to_lab(sx, sy, y0, g.cf.dtyi[pixel_mask], g.cf.omega[pixel_mask])
        
        new_pars = clean_pars.copy()
        new_pars['distance'] = new_pars['distance'] - lx
        
        tth, eta = ImageD11.transform.compute_tth_eta(
                        (g.cf.sc[pixel_mask], g.cf.fc[pixel_mask]),
                        **new_pars)

        gve = ImageD11.transform.compute_g_vectors(tth,
                            eta,
                            g.cf.omega[pixel_mask],
                            new_pars['wavelength'],
                            wedge=new_pars['wedge'],
                            chi=new_pars['chi'])
        
        # save gvecs
        gvecs_per_point[ri, rj] = gve.T
    
    # concatenate all gvecs together to compute refined ubi
    all_gvecs = np.vstack([gve for gve in gvecs_per_point.values()])

    ubifit = g.ubi.copy()
    _ = cImageD11.score_and_refine(ubifit, all_gvecs, peak_assign_tol)
    g.set_ubi(ubifit)

    # now iterate through each pixel position
    for mi, mj, mk in g.pixel_support_coords:
        ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
        # we already have the recomputed g-vectors
        gvecs_here = gvecs_per_point[ri, rj]
        ubifit = g.ubi.copy()
        _ = cImageD11.score_and_refine(ubifit, gvecs_here, peak_assign_tol)
        per_pixel_ubis[ri, rj] = (ginc, ubifit)

In [ ]:
# validate masking

fig, ax = plt.subplots()
ax.scatter(cf_2d_strong.omega[cf_2d_strong.grain_id == ginc], cf_2d_strong.dty[cf_2d_strong.grain_id == ginc], label='cf_2d_strong assignments')
ax.scatter(g.cf.omega, g.cf.dty, label='grain cf - spatially filtered')
ax.scatter(g.cf.omega[pixel_mask], g.cf.dty[pixel_mask], label='pixel mask')
ax.invert_yaxis()
ax.legend()
plt.show()

In [ ]:
# determine a UBI per pixel

ubi_map = np.empty((recon_shape + (3,3)))
ubi_map.fill(np.nan)
for pxi in tqdm(range(recon_shape[0])):
    for pxj in range(recon_shape[1]):
        try:
            graininc, this_ubi = per_pixel_ubis[pxi, pxj]
            ubi_map[pxi, pxj, :, :] = this_ubi
        except KeyError:
            continue

In [ ]:
ubi_map_tmap = tensor_map.recon_order_to_map_order(ubi_map)

In [ ]:
tensor_map.add_map('UBI_refined', ubi_map_tmap)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(tensor_map.UBI[0, :, :, 0, 0])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(tensor_map.UBI_refined[0, :, :, 0, 0])
plt.show()

In [ ]:
tensor_map.UBI = tensor_map.UBI_refined
tensor_map.keys()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(tensor_map.unitcell[0, :, :, 0])
plt.show()

In [ ]:
rel_vol_strain_map = np.full(tensor_map.phase_ids.shape, np.nan, float)
abs_vol_strain_map = np.full(tensor_map.phase_ids.shape, np.nan, float)

NZ, NY, NX = tensor_map.shape

for mi in range(NZ):
    for mj in range(NY):
        for mk in range(NX):
            # get the unitcell at this position
            unitcell_px = tensor_map.unitcell[mi, mj, mk]
            # get the grain label at this position
            label_px = tensor_map.labels[mi, mj, mk]
            # get the grain at this position
            grain_px = grains[label_px]
            unitcell_px_meanlength = np.mean(unitcell_px[:3])
            grain_px_meanlength = np.mean(grain_px.unitcell[:3])
            ref_ucell_meanlength = np.mean(ds.phases.unitcells[phase_str].lattice_parameters[:3])
            rel_vol_strain = (unitcell_px_meanlength - grain_px_meanlength) / grain_px_meanlength
            abs_vol_strain = (unitcell_px_meanlength - ref_ucell_meanlength) / ref_ucell_meanlength
            rel_vol_strain_map[mi, mj, mk] = rel_vol_strain
            abs_vol_strain_map[mi, mj, mk] = abs_vol_strain

tensor_map.add_map('rel_vol_strain', rel_vol_strain_map)
tensor_map.add_map('abs_vol_strain', abs_vol_strain_map)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(tensor_map.rel_vol_strain[0, :, :]*1e3, cmap='RdBu')
plt.colorbar(im)
plt.title('Relative volumetric strain (1e-3)')
plt.show()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(tensor_map.abs_vol_strain[0, :, :]*1e3, cmap='RdBu')
plt.colorbar(im)
plt.title('Absolute volumetric strain (1e-3)')
plt.show()

In [ ]:
# write the TensorMap to disk too

tensor_map.to_h5(ds.grainsfile, h5group='TensorMap_' + phase_str + '_refined')

In [ ]:
ds.save()

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# Now that we're happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}
    
# now we have our samples_dict, we can process our data:

nthreads = len(os.sched_getaffinity(os.getpid()))

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if not os.path.exists(ds.grainsfile):
            print(f"Missing grains file for {dataset} in sample {sample}, skipping")
            continue
            
        # check grains file for existance of output, skip if it's there
        with h5py.File(ds.grainsfile, "r") as hin:
            if 'TensorMap_' + phase_str + '_refined' in hin.keys():
                print(f"Already reconstructed {dataset} in {sample}, skipping")
                continue
        
        ds.phases = ds.get_phases_from_disk()
        
        cf_2d = ds.get_cf_2d()
        ds.update_colfile_pars(cf_2d, phase_name=phase_str)

        grainsinos = read_h5(ds.grainsfile, ds, phase_str)
        grains = [gs.grain for gs in grainsinos]
        
        tensor_map = TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + phase_str)
        
        cf_2d_strong = select_ring_peaks_by_intensity(cf_2d, frac=cf_2d_strong_frac, dsmax=cf_2d.ds.max())
        y0 = grainsinos[0].recon_y0
        recon_shape = grainsinos[0].recons["astra"].shape

        dtyi = geometry.dty_to_dtyi(cf_2d_strong.dty, ystep=ds.ystep)
        cf_2d_strong.addcolumn(dtyi, "dtyi")
        
        utils.assign_peaks_to_grains(grains, cf_2d_strong, tol=peak_assign_tol)

        per_pixel_ubis = {}
        grain_lut = {}

        clean_pars = cf_2d_strong.parameters.get_parameters()

        for ginc, g in enumerate(tqdm(grains[:])):
            grain_lut[ginc] = g

            # work out what 2D peaks this grain claims
            g.mask_2d_strong = cf_2d_strong.grain_id == ginc

            # get the 2D peaks for this grain only
            # for now this will become the columnfile for the grain
            g.cf = cf_2d_strong.copyrows(g.mask_2d_strong)

            # work out what pixels in the sample this grain claims
            g.pixel_support_mask = tensor_map.labels == ginc
            g.pixel_support_coords = np.argwhere(g.pixel_support_mask == True)

            if len(g.pixel_support_coords) == 0:
                continue

            # right now, g.cf is just from the peak assignment
            # could contain peaks that come from the wrong place in the sample
            # solution: combine these 2 masks together
            # isolates 2D peaks that index this grain and come from the grain spatially

            # this will mask g.cf based on pixel position
            new_mask_2d_strong = np.zeros(g.cf.nrows).astype(bool)

            # iterate through the pixel positions
            for mi, mj, mk in g.pixel_support_coords:
                # get a peak mask associated with this pixel position in the sample
                ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
                mask = geometry.dtyimask_from_recon(ri, rj, g.cf.omega, g.cf.dtyi, recon_shape=recon_shape, ystep=ds.ystep, y0=y0)

                # add it to the mask (| is OR)
                new_mask_2d_strong |= mask

            g.cf = g.cf.copyrows(new_mask_2d_strong)

            gvecs_per_point = {}
            all_tth_old = []
            all_tth_new = []
            all_omega = []
            for mi, mj, mk in g.pixel_support_coords:
                ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
                # get translation in the sample frame
                sx, sy = geometry.recon_to_sample(ri, rj, recon_shape=recon_shape, ystep=ds.ystep)

                # get a peak mask associated with this pixel position in the sample
                pixel_mask = geometry.dtyimask_from_recon(ri, rj, g.cf.omega, g.cf.dtyi, recon_shape=recon_shape, ystep=ds.ystep, y0=y0)

                # get the x translation of the peak in the lab reference frame

                lx, _ = geometry.sample_to_lab(sx, sy, y0, g.cf.dtyi[pixel_mask], g.cf.omega[pixel_mask])

                new_pars = clean_pars.copy()
                new_pars['distance'] = new_pars['distance'] - lx

                tth, eta = ImageD11.transform.compute_tth_eta(
                                (g.cf.sc[pixel_mask], g.cf.fc[pixel_mask]),
                                **new_pars)

                gve = ImageD11.transform.compute_g_vectors(tth,
                                    eta,
                                    g.cf.omega[pixel_mask],
                                    new_pars['wavelength'],
                                    wedge=new_pars['wedge'],
                                    chi=new_pars['chi'])

                # save gvecs
                gvecs_per_point[ri, rj] = gve.T

            # concatenate all gvecs together to compute refined ubi
            all_gvecs = np.vstack([gve for gve in gvecs_per_point.values()])

            ubifit = g.ubi.copy()
            _ = cImageD11.score_and_refine(ubifit, all_gvecs, peak_assign_tol)
            g.set_ubi(ubifit)

            # now iterate through each pixel position
            for mi, mj, mk in g.pixel_support_coords:
                ri, rj = tensor_map.map_index_to_recon(mj, mk, yshape=tensor_map.shape[1])
                # we already have the recomputed g-vectors
                gvecs_here = gvecs_per_point[ri, rj]
                ubifit = g.ubi.copy()
                _ = cImageD11.score_and_refine(ubifit, gvecs_here, peak_assign_tol)
                per_pixel_ubis[ri, rj] = (ginc, ubifit)

        ubi_map = np.empty((recon_shape + (3,3)))
        ubi_map.fill(np.nan)
        for pxi in tqdm(range(recon_shape[0])):
            for pxj in range(recon_shape[1]):
                try:
                    graininc, this_ubi = per_pixel_ubis[pxi, pxj]
                    ubi_map[pxi, pxj, :, :] = this_ubi
                except KeyError:
                    continue
        
        ubi_map_tmap = tensor_map.recon_order_to_map_order(ubi_map)
        tensor_map.UBI = tensor_map.UBI_refined
        eul = tensor_map.euler
        
        rel_vol_strain_map = np.full(tensor_map.phase_ids.shape, np.nan, float)
        abs_vol_strain_map = np.full(tensor_map.phase_ids.shape, np.nan, float)

        NZ, NY, NX = tensor_map.shape

        for mi in range(NZ):
            for mj in range(NY):
                for mk in range(NX):
                    # get the unitcell at this position
                    unitcell_px = tensor_map.unitcell[mi, mj, mk]
                    # get the grain label at this position
                    label_px = tensor_map.labels[mi, mj, mk]
                    # get the grain at this position
                    grain_px = grains[label_px]
                    unitcell_px_meanlength = np.mean(unitcell_px[:3])
                    grain_px_meanlength = np.mean(grain_px.unitcell[:3])
                    ref_ucell_meanlength = np.mean(ds.phases.unitcells[phase_str].lattice_parameters[:3])
                    rel_vol_strain = (unitcell_px_meanlength - grain_px_meanlength) / grain_px_meanlength
                    abs_vol_strain = (unitcell_px_meanlength - ref_ucell_meanlength) / ref_ucell_meanlength
                    rel_vol_strain_map[mi, mj, mk] = rel_vol_strain
                    abs_vol_strain_map[mi, mj, mk] = abs_vol_strain

        tensor_map.add_map('rel_vol_strain', rel_vol_strain_map)
        tensor_map.add_map('abs_vol_strain', abs_vol_strain_map)
        
        tensor_map.to_h5(ds.grainsfile, h5group='TensorMap_' + phase_str + '_refined')

        ds.save()

print("Done!")